In [1]:
#coding=utf-8
import pandas as pd
import numpy as np

train_csv =  "/Users/daishuyu/Downloads/互联网数据挖掘作业3-1900013074-戴舒羽/dataset/train.csv"
val_csv = "/Users/daishuyu/Downloads/互联网数据挖掘作业3-1900013074-戴舒羽/dataset/val.csv"
test_csv = "/Users/daishuyu/Downloads/互联网数据挖掘作业3-1900013074-戴舒羽/dataset/test.csv"

**一种比较naive的方式：直接拆字，比较是否在里面-直观，一些情况下有效**


In [2]:
cz_dict = {}#key:字形  value:拆出来的部首/笔画，可能有多种拆字方法，用'|'分开
with open("chaizi_jt.txt") as f:
    lines = f.readlines()      #读取全部内容 ，并以列表方式返回
    for line in lines:
        line = line.replace('\t',' ').strip('\n').split(' ')
        char = line[0]
        char_cz = line[1:]
        char_cz_set = set(char_cz)
        char_cz = list(char_cz_set)
        cz_dict[char] = char_cz 
print(list(cz_dict.items())[0:1])

[('丁', ['亅', '一'])]


In [3]:
def get_query_cz(query):
    query_cz = []

    for char in query:    
        if char in cz_dict:
            query_cz.extend(cz_dict[char])
        query_cz.append(char)
        query_cz.append(char)
        query_cz.append(char)
    return query_cz

print(get_query_cz(query = '走西口'))

['疋', '十', '走', '走', '走', '一', '口', '囗', '四', '儿', '西', '西', '西', '冂', '乛', '一', '丨', '口', '口', '口']


In [4]:
def cal_cz_best(query,c):
    q_cz = get_query_cz(query)
    c_scores = []
    for i in range(5):
        c_cz = get_query_cz(c[i])
        c_scores.append(0)
        for ch in q_cz:
            if ch in c_cz:
                c_scores[i] += c_cz.count(ch)
    return c_scores,c_scores.index(max(c_scores))

def test_cal_cz_best(query,c):
    q_cz = get_query_cz(query)
    c_scores = []
    for i in range(5):
        c_cz = get_query_cz(c[i])
        c_scores.append(0)
        for ch in q_cz:
            if ch in c_cz:
                c_scores[i] += c_cz.count(ch)          
    return c_scores,c_scores.index(max(c_scores))

#正确示例
print(cal_cz_best(query='皇',c=['赛雪','如瓷','白玉无瑕','红装素裹','吹弹得破']))
print(cal_cz_best(query='铆',c=['梅花鹿','旺旺','琉璃摆件','金兔','小貔貅']))
print(cal_cz_best(query='明',c=['一朝一夕','日积月累','三天打鱼，两天晒网','潜移默化','慢慢的我']))#干扰项“朝”，通过增大字本身权重保证正确性
print(cal_cz_best(query='昔',c=['心惊胆寒','突如其来','意料之外','人仰马翻','措手不及']))#干扰项“胆”，通过增大字本身权重保证正确性

#错误示例
##并列-给出候选答案
print(cal_cz_best(query='几',c=['忘乎所以','厚颜无耻','饥不择食','人面兽心','劫色']))
print(cal_cz_best(query='忘',c=['感恩戴德','死心塌地','委曲求全','不计前嫌','动了情']))#亡和死在语义上接近
##字形方法无法处理
print(cal_cz_best(query='十',c=['层叠','坑坑洼洼','狭长','密密麻麻','纵横交错']))#语义解释字形

([0, 0, 4, 0, 0], 2)
([0, 0, 0, 3, 0], 3)
([2, 6, 1, 0, 0], 1)
([1, 0, 0, 0, 3], 4)
([2, 1, 5, 5, 0], 2)
([3, 3, 0, 0, 1], 0)
([0, 0, 0, 0, 0], 0)


在训练集和验证集上检测准确率

In [5]:
train_df = pd.read_csv(train_csv,header=0,sep=',') #header=None表示头部为空，sep=','表示数据间使用逗号作为分隔符
print(train_df.shape)
val_df = pd.read_csv(val_csv,header=0,sep=',') #header=None表示头部为空，sep=','表示数据间使用逗号作为分隔符
print(val_df.shape)

ls=[]
lssmall = []
lssent = []

for i in range(train_df.shape[0]):
    string = train_df.iloc[i,0]
    index = string.find(' ')
    if index == -1 or index == 0:
        index = string.find('（')
    string = string[0:index]
    train_df.iloc[i,0] = string
    if len(string) <= 1:
        ls.append(i)
    elif '，' not in string:
        lssmall.append(i)
    else:
        lssent.append(i)
        
t = train_df.loc[ls]
print(t.shape,t.head(1))
t2 = train_df.loc[lssmall]
print(t2.shape,t2.head(1))
t3 = train_df.loc[lssent]
print(t3.shape,t3.head(1))

(4000, 7)
(500, 7)
(79, 7)   riddle choice0 choice1 choice2 choice3 choice4  label
9      皇      赛雪      如瓷    白玉无瑕    红装素裹    吹弹得破      2
(2434, 7)    riddle choice0 choice1 choice2 choice3 choice4  label
0  火把接力传递    付之一炬    流离失所      焚毁      掠走    苦心经营      0
(1487, 7)                              riddle choice0 choice1 choice2 choice3 choice4  \
4  不是鱼鳔打个结，不是泥鳅泥里歇，不是蚕虫又吐丝，不是蜂窝多洞穴。      冬笋       藕      茨菇      荸荠      茭白   

   label  
4      1  


In [6]:
def test_on_dataset(dataset):
    accuracy = 0
    accurate_index = []
    for i in range(dataset.shape[0]):
        row = dataset.iloc[i]
        pred_scores, pred = cal_cz_best(row.riddle,[row.choice0,row.choice1,row.choice2,row.choice3,row.choice4])
        if int(row.label) == pred:
            accuracy += 1
        else:
            accurate_index.append(i)
    return accuracy,dataset.shape[0],accuracy/dataset.shape[0]#dataset.iloc[accurate_index],

In [7]:
print(test_on_dataset(t))
print(test_on_dataset(t2))
print(test_on_dataset(t3))
print(test_on_dataset(train_df))
print(test_on_dataset(val_df))

(58, 79, 0.7341772151898734)
(719, 2434, 0.2953985209531635)
(362, 1487, 0.24344317417619368)
(1139, 4000, 0.28475)
(131, 500, 0.262)


在测试集上测试

In [8]:
test_df = pd.read_csv(test_csv,header=0,sep=',') #header=None表示头部为空，sep=','表示数据间使用逗号作为分隔符
print(test_df.shape)

ls=[]
lssmall = []
lssent = []

for i in range(test_df.shape[0]):
    string = test_df.iloc[i,0]
    index = string.find(' ')
    if index == -1 or index == 0:
        index = string.find('（')
    string = string[0:index]
    test_df.iloc[i,0] = string
    if len(string) <= 1:
        ls.append(i)
    elif '，' not in string:
        lssmall.append(i)
    else:
        lssent.append(i)
        

t = test_df.loc[ls]
print(t.shape,t.head(1))
t2 = test_df.loc[lssmall]
print(t2.shape,t2.head(1))
t3 = test_df.loc[lssent]
print(t3.shape,t3.head(1))

(2000, 6)
(35, 6)    riddle choice0 choice1 choice2 choice3 choice4
23      女   元和圣德诗    如出一口    古之遗直      异哉    杂然相许
(1184, 6)   riddle choice0 choice1 choice2 choice3 choice4
0  三思而后行     随大流   跟着感觉走    及时行乐    勇敢一点   做自己就好
(781, 6)              riddle choice0 choice1 choice2 choice3 choice4
1  黑缎子，白夹里，养儿养女泥窝里。     树袋熊      黑熊     蜘蛛蟹      知了      狗熊


In [9]:
def pred_on_dataset(dataset):
    preds = []
    for i in range(dataset.shape[0]):
        row = dataset.iloc[i]
        pred_scores, pred = test_cal_cz_best(row.riddle,[row.choice0,row.choice1,row.choice2,row.choice3,row.choice4])
        preds.append(pred)
    return preds

In [10]:
res1 = pred_on_dataset(t)
res2 = pred_on_dataset(t2)
res3 = pred_on_dataset(t3)
res = pred_on_dataset(test_df)
